# Dataset and Problem Statement

The MNIST Dataset used in this use case consists of 60'000 data samples representing images of digits between 0 and 9. Each image sample has 784 attributes (shape 28x28x1). This dataset is available on [Kaggle](https://www.kaggle.com/datasets/oddrationale/mnist-in-csv). We already preprocessed the data and saved it in the `data/MnistDataset` folder. It is composed of 50'000 training samples and 10'000 testing samples.

Each attribute is a pixel value ranging from 0 to 255. It's not necessary to normalize the data as it is done during the CNN training process.

**Problem Statement:** Our objective is to build a robust CNN classifier capable of accurately classifying the images among the 10 classes. By leveraging deep learning techniques and Fidex algorithms, we aim to not only achieve high classification performance but also gain insights into the attributes that contribute to the classification decisions.

We'll start by importing all libraries. A warning might appear, but there's no need to worry about it.

In [1]:
import os, sys
from dimlpfidex.fidex import fidex, fidexGloRules, fidexGloStats, fidexGlo
from trainings.cnnTrn import cnnTrn

/home/jeanmarc/dimlpfidex/notebooks/.venv/lib/python3.10/site-packages/numpy/core/getlimits.py:542: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


# Model training

We train the MNIST dataset with a CNN. We use the cnnTrn module to do so. First, let's look at the parameters :

In [2]:
status = cnnTrn()

Usage: 
--train_data_file <str> --test_data_file <str> --original_input_size <pair<int [1, inf[>> --nb_channels <int [1,inf[> --model <{small, large, vgg, resnet}> --data_format <{normalized_01, classic, other}> --nb_classes <int [1,inf[> [-h, --help] [--json_config_file <str>] [--root_folder <str>] [--train_class_file <str>] [--test_class_file <str>] [--train_valid_pred_outfile <str>] [--test_pred_outfile <str>] [--valid_ratio <float ]0,inf[>] [--valid_data_file <str>] [--valid_class_file <str>] [--weights_outfile <str>] [--stats_file <str>] [--console_file <str>] [--nb_epochs <int [1,inf[>] [--nb_quant_levels <int [3,inf[>] [--K <float ]0,inf[>] [--model_input_size <pair<int [1, inf[>>] [--seed <{int [0,inf[}>]

This is a parser for cnnTrn


Parameters:

  ---------------------------------------------------------------------

  Warning! The files are located with respect to the root folder dimlpfidex.

  The arguments can be specified in the command or in a json configuration file wi

To configure our training, we will use this configuration file, located at `data/MnistTemplates/config_cnn.json`:

```json
{
    "root_folder": "data/MnistDataset",
    "train_data_file": "trainData.txt",
    "train_class_file": "trainClass.txt",
    "test_data_file": "testData.txt",
    "test_class_file": "testClass.txt",
    "valid_ratio": 0.1,
    "original_input_size": "[28,28]",
    "data_format": "classic",
    "nb_channels": 1,
    "nb_classes": 10,
    "model": "small"
}
```

> A detailed list of parameters is available [here](https://hes-xplain.github.io/documentation/algorithms/trainings/cnnTrn/). If you want to create your own configuration file, we recommend using our [configuration file creator](https://hes-xplain.github.io/documentation/gui/)
TODO : ajouter partout dans tous les notebooks le lien vers les listes de paramètres.


We start by training our model. As it may take some time, the training has already been done. If you wish to train it yourself, you can uncomment and run the next instruction :

In [3]:
#res = cnnTrn('--json_config_file data/MnistTemplates/config_cnn.json')
#if (res == 0):
#    print("cnnTrn done")

You can check all outputs inside the `data/MnistDataset` folder.

Inside `stats.txt` you can see the train and test accuracy.

There are also the train predictions, test predictions, and the weights.

# Local rules generation - Fidex

Fidex is an algorithm used in classification problems that allows us to obtain a rule explaining the decision class of the model for a given test sample. It enables us to grasp the model's decision-making process and to better understand the importance of each parameter in discerning the nature of the tumor, distinguishing between benign and malignant cases.

Now we can generate some local rules to explain the models' results. We can start with launching Fidex on one test sample. This will generate a rule explaining the sample locally. It is local because the algorithm searches a rule only for one sample.
Fidex is located in the fidex module. Let's take a look at the parameters :

In [4]:
res = fidex("--help")


---------------------------------------------------------------------

Warning! The files are located with respect to the root folder dimlpfidex.
The arguments can be specified in the command or in a json configuration file with --json_config_file your_config_file.json.

----------------------------

Required parameters:

--train_data_file <str>       Path to the file containing the train portion of the dataset
--train_class_file <str>      Path to the file containing the train true classes of the dataset, not mandatory if classes are specified in train data file
--train_pred_file <str>       Path to the file containing predictions on the train portion of the dataset
--test_data_file <str>        Path to the file containing the test sample(s) data, prediction (if no --test_pred_file) and true class(if no --test_class_file)
--weights_file <str>          Path to the file containing the trained weights of the model (not mandatory if a rules file is given with --rules_file)
--rules_file <

We see that we need the number of attributes and classes, the train and test data and class files, as well as train and test predictions, weights file name, and rule output file name where the rules will be saved.<br>
We specify as well the saved folder and the attributes.<br>
We also need to specify the normalization file obtained from training, to denormalize the values in the generated rule, otherwise the values will be normalized and impossible to interpret.<br>

To see how it works, we launch it with just one sample using the configuration file `data/MnistTemplates/config_fidex.json`, and we have saved beforehand the test data sample with its class and predictions in the file `data/MnistDataset/testDataSample.txt`. **It should take about 1 minute**.

In [5]:
res = fidex("--json_config_file data/MnistTemplates/config_fidex.json")

Parameters list:
 - train_data_file                                                      data/MnistDataset/trainData.txt
 - train_pred_file                                                      data/MnistDataset/predTrain.out
 - train_class_file                                                    data/MnistDataset/trainClass.txt
 - test_data_file                                                  data/MnistDataset/testDataSample.txt
 - rules_outfile                                                             data/MnistDataset/rule.rls
 - root_folder                                                                        data/MnistDataset
 - weights_file                                                           data/MnistDataset/weights.wts
 - nb_attributes                                                                                    784
 - nb_classes                                                                                        10
 - nb_quant_levels                             

You can see the walkthrough of the algorithm and the rule extracted. The rule is also saved in the rule.rls file. The Xi terms reprensent the ith pixel of the image (or ith attribute). With the rule, we see also the covering size of the rule on the training set, the fidelity, the accuracy, and the confidence of the rule. The confidence shows how much the rule is confident with his choices, with respect to the prediction values.<br>

# Global ruleSet generation - FidexGlo
We've seen how to compute a rule that explains the decision of the model for a specific sample. Now, we will generalize a ruleset that characterizes the whole train dataset. That means that for each training sample, there is a rule in the set of rules that explains the model's decision for this sample. We will use this global ruleset to explain the results obtained on new test samples. If there is a rule of the ruleset corresponding to the sample, we take this one and get a global explanation for the sample. If there is none, we call Fidex and only have a local explanation.<br>

To get the ruleSet we execute fidexGloRules which is located in the fidex module. Here are the possible parameters :

In [6]:
res = fidexGloRules("--help")


---------------------------------------------------------------------

Warning! The files are located with respect to the root folder dimlpfidex.
The arguments can be specified in the command or in a json configuration file with --json_config_file your_config_file.json.

----------------------------

Required parameters:

--train_data_file <str>       Path to the file containing the train portion of the dataset
--train_class_file <str>      Path to the file containing the train true classes of the dataset, not mandatory if classes are specified in train data file
--train_pred_file <str>       Path to the file containing predictions on the train portion of the dataset
--weights_file <str>          Path to the file containing the trained weights of the model (not mandatory if a rules file is given with --rules_file)
--rules_file <str>            Path to the file containing the trained rules to be converted to hyperlocus (not mandatory if a weights file is given with --weights_file)
--gl

We use nearly the same parameters as for Fidex but we only need train data. We need to choose a heuristic for fidexGloRules, we choose the optimal to get better results.<br>
The process is very long and can last several days, so we already computed it beforehand using the configuration file `data/MnistTemplates/config_fidexGloRules.json`. The ruleset is available in the file `data/MnistDataset/globalRules.txt`.<br>
If you want to launch it, and you have several processors available, you should add the parameter nb_threads with the number of processors that you want to use, it can speed up the process a lot. If you want to accelerate the process even more, you can use some dropout, the algorithm will randomly skip some dimensions or some hyperplans. For example, you can put: -d 0.9 -h 0.9, to skip half dimensions and half hyperplans, which should be about 4 times faster. You just need to uncomment the next lines :

In [7]:
#res = fidexGloRules('--json_config_file data/MnistTemplates/config_fidexGloRules.json')  
#if (res == 0):
#    print("FidexGloRules done")

We have generated about 5000 rules explaining the whole train dataset. You can see at the top of the globalRules file the number of rules, the mean covering number per rule, and the mean number of antecedents. Here is an example of a rule that you may obtain:<br>

Rule 1: X323>=219.3 X517>=198.9 X180<5.1 X492<25.5 X272<25.5 X347<10.2 X576<20.4 X236<30.6 X436<132.6 X541<30.6 -> class 1 <br>
   Train Covering size : 2156 <br>
   Train Fidelity : 1 <br>
   Train Accuracy : 1 <br>
   Train Confidence : 0.999678 <br>

This rule is the first rule, which means that it's the rule with the maximum covering. Here, 2156 train samples verify this rule. It is 100% fidel with the model and has about 100% train accuracy.
This rule says that if the 323rd pixel of the image is greater than 219.3 and if the other antecedents are verified, then this is an image on the digit 1. And this rule is accurate, on train test, at 100% and has 99% of confidence.<br>

Now, we can see some global statistics on the test set, and also some statistics directly on each rule. So we will see the test accuracy on this rule.<br>
We execute fidexGloStats which is located in the fidex module. First, let's check the parameters :

In [8]:
res = fidexGloStats("--help")


---------------------------------------------------------------------

Warning! The files are located with respect to the root folder dimlpfidex.
The arguments can be specified in the command or in a json configuration file with --json_config_file your_config_file.json.

----------------------------

Required parameters:

--test_data_file <str>        Path to the file containing the test portion of the dataset
--test_class_file <str>       Path to the file containing the test true classes of the dataset, not mandatory if classes are specified in test data file
--test_pred_file <str>        Path to the file containing predictions on the test portion of the dataset
--global_rules_file <str>     Path to the file containing the global rules obtained with fidexGloRules algorithm.
--nb_attributes <int [1,inf[> Number of attributes in the dataset
--nb_classes <int [2,inf[>    Number of classes in the dataset

----------------------------

Optional parameters: 

--json_config_file <str>      

We need the test files(data, prediction, class), the rules file, and the number of attributes and classes. We choose to save the results in the file fidexGloStats.txt. With the parameter global_rules_outfile we can generate the statistics on rules which will modify the rules file. If you want to keep the first ruleSet unchanged, you should give another name. So here is the command using the configuration file `data/MnistTemplates/config_fidexGloStats.json`:

In [9]:
res = fidexGloStats("--json_config_file data/MnistTemplates/config_fidexGloStats.json")

Parameters list:
 - test_data_file                                                        data/MnistDataset/testData.txt
 - test_pred_file                                                        data/MnistDataset/predTest.out
 - test_class_file                                                      data/MnistDataset/testClass.txt
 - global_rules_outfile                                  data/MnistDataset/globalRulesWithTestStats.rls
 - global_rules_file                                                  data/MnistDataset/globalRules.txt
 - root_folder                                                                        data/MnistDataset
 - stats_file                                                       data/MnistDataset/fidexGloStats.txt
 - nb_attributes                                                                                    784
 - nb_classes                                                                                        10
 - positive_class_index                        

Here you can see the global statistics on the test set. We have about 97% fidelity, which is good, and a rule accuracy(96.6%) about 3% lower than the model accuracy(99.3%). So the rules seem to classify a bit worse. The explainability rate is the percentage of samples for which we can find a rule in the rules set. For the others, we need to execute Fidex (this is the default rule rate). In our case, we had more than a 95% explainability rate, so only in 5% of cases do we need to compute Fidex. Each rule can activate many rules. Here on average, a sample activates 8 correct rules and 0.2 wrong rules. A wrong rule is a rule with which the model doesn't agree. For example, if the rule says malign and the model says benign. Something interesting is the model test accuracy when rules and model agree. You can see that, generally, the accuracy increases if we consider samples where rules and model agree, and increases even more if we take only the activated rules (when there are no activated rules, we choose the model prediction). That means that the rules confirm well the model decision, but when no rule is found, the model decision may as well be wrong. <br>

In globalRulesWithTestStats file, you can now see the statistics of rules on the test set. Here is the same rule as before that we have now:<br>

Rule 1: X323>=219.3 X517>=198.9 X180<5.1 X492<25.5 X272<25.5 X347<10.2 X576<20.4 X236<30.6 X436<132.6 X541<30.6 -> class 1 <br>
   Train Covering size : 2156 --- Test Covering size : 370 <br>
   Train Fidelity : 1 --- Test Fidelity : 0.997297 <br>
   Train Accuracy : 1 --- Test Accuracy : 1 <br>
   Train Confidence : 0.999678 --- Test Confidence : 0.997311 <br>

We see that the rule no longer always agrees with the model, only in 99.7% of cases. However, the rule accuracy is perfect. That means that the rule is very good in reality, with 100% of correct classification.

# Explanation and image generation

Now we get the explanations on a few test samples. We use the files `testDataSamples`, `testClassSamples`, and `testPredSamples` containing 10 test samples. <br>
We launch it like this using the configuration file `data/MnistTemplates/config_fidexGlo.json` :

In [10]:
res = fidexGlo("--json_config_file data/MnistTemplates/config_fidexGlo.json")
if res == 0:
    print("Explanations generated") 

Explanations generated


The explanations for each test sample can then be found in the file `data/MnistDataset/explanations.txt`. We find a global rule for every of those 10 samples. <br>
Now, we parse the explanation file to get the first exaplanation rule of each sample and we generate the test images with colored pixels where the rule is activated.

In [11]:
import re
import numpy as np
from PIL import Image

images = []
test_data = "data/MnistDataset/testDataSamples.txt"
with open(test_data, "r") as my_file:
    for line in my_file:
        images.append(line.strip().split(" "))

explanation_file = "data/MnistDataset/explanations.txt"
pattern = r'X(\d+)\s*([<>]=?)\s*([\d.]+)' # Regular expression pattern to match antecedents
rules = []
with open(explanation_file, "r") as my_file:
    for line in my_file:
        if line.startswith("R1: "):
            rules.append(line.strip())
        if line.startswith("Local rule"):
            # Search next non empty line
            next_line = next(my_file, '').strip()
            while not next_line:
                next_line = next(my_file, '').strip()
            rules.append(next_line)
    my_file.close()

# Find all matches in the input string
for id_sample in range(len(rules)):
    antecedents = []
    matches = re.findall(pattern, rules[id_sample])

    # Process each match and store in antecedents
    for match in matches:
        attribute, inequality, value = match
        antecedent = {
            "attribute": int(attribute),
            "inequality": inequality,
            "value": float(value)
        }
        antecedents.append(antecedent)

    colorimage = [[v,v,v] for v in images[id_sample]]
    for antecedent in antecedents:
                if antecedent["inequality"] == "<":
                    colorimage[antecedent["attribute"]]=[255,0,0]
                else:
                    colorimage[antecedent["attribute"]]=[0,255,0]

    colorimage_array = np.array(colorimage).reshape(28, 28, 3)
    colorimage = Image.fromarray(colorimage_array.astype('uint8'))
    image_path = 'data/MnistDataset/images/img_'+ str(id_sample) + '_out.png'
    colorimage.save(image_path)
    colorimage.show()


You can observe the 10 images. To see them better, open them in the folder `data/MnistDataset/images`. The red dots indicate pixels where the rule requires the value to be below a certain threshold, while the green dots represent pixels where the value must be above a threshold. You can observe which pixels are used by the model to decide which digit it is.

# Conclusion

Throughout this notebook, we've learned how to use cnnTrn to train a dataset and get some explaining rules, and we got into the different Fidex algorithms to get some local and global explanations, and some statistics on the model's decisions. We used the example of the MNIST dataset and we saw that the algorithms perform very well. It gives some good explanations of the model's decision. You can try with any image dataset you want, you just have to remember to transform the data in a proper format. <be>

We didn't speak about every single parameter, you can try to change every hyperparameter you want to see how it goes. A parameter that can be useful is the seed. It allows you to remove the randomness of the algorithms and get the same result in each execution for the same parameters and data.<br>

To go further, you can see the notebook on the Cracks dataset which is another application of CNNs on image classification [INSERER LIEN NOTEBOOK CRACKS].

# Références

Article DimlpBT de Guido
Article Fidex de Guido (à venir)